In case our environment sucks and we have to bootstrap libraries ourselves.

In [ ]:
import sys, subprocess
# subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras', 'opencv-python', 'tensorflow'])

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.models import Sequential
from keras.saving import load_model

print("Loaded all libraries")

Boostrap dataset

In [ ]:
# prompt: write python code to do this command curl -L -o ~/Downloads/archive.zip\
# https://www.kaggle.com/api/v1/datasets/download/jessicali9530/stanford-dogs-dataset
# also unzip it

import os
import subprocess

if not os.path.exists("data"):
  print("Dataset not downloaded, downloading...")

  !curl -L -o archive.zip https://www.kaggle.com/api/v1/datasets/download/jessicali9530/stanford-dogs-dataset
  !unzip -q archive.zip -d data
  !rm archive.zip

In [ ]:
from pathlib import Path
fpath = str(Path("data/images/Images").resolve().absolute())
random_seed = 42

categories = os.listdir(fpath)
categories = categories[:20]
print("List of categories = ",categories,"\n\nNo. of categories = ", len(categories))

In [ ]:
def load_images_and_labels(categories):
    img_lst=[]
    labels=[]
    for index, category in enumerate(categories):
        for image_name in os.listdir(fpath+"/"+category):
            img = cv2.imread(fpath+"/"+category+"/"+image_name)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            img_array = Image.fromarray(img, 'RGB')

            #resize image to 224 x 224 because the input image resolution for ResNet50 is 224 x 224
            resized_img = img_array.resize((224, 224))

            img_lst.append(np.array(resized_img))

            labels.append(index)
    return img_lst, labels

images, labels = load_images_and_labels(categories)
print("No. of images loaded = ",len(images),"\nNo. of labels loaded = ",len(labels))
print(type(images),type(labels))

In [ ]:
images = np.array(images)
labels = np.array(labels)

print("Images shape = ",images.shape,"\nLabels shape = ",labels.shape)
print(type(images),type(labels))

In [ ]:
def display_rand_images(images, labels):
    plt.figure(1 , figsize = (19 , 10))
    n = 0
    for i in range(9):
        n += 1
        r = np.random.randint(0 , images.shape[0] , 1)

        plt.subplot(3 , 3 , n)
        plt.subplots_adjust(hspace = 0.3 , wspace = 0.3)
        plt.imshow(images[r[0]])

        plt.title('Cute Dog : {}'.format(labels[r[0]]))
        plt.xticks([])
        plt.yticks([])

    plt.show()

display_rand_images(images, labels)

In [ ]:
#1-step in data shuffling

#get equally spaced numbers in a given range
n = np.arange(images.shape[0])
print("'n' values before shuffling = ",n)

#shuffle all the equally spaced values in list 'n'
np.random.seed(random_seed)
np.random.shuffle(n)
print("\n'n' values after shuffling = ",n)

In [ ]:
#2-step in data shuffling

#shuffle images and corresponding labels data in both the lists
images = images[n]
labels = labels[n]

print("Images shape after shuffling = ",images.shape,"\nLabels shape after shuffling = ",labels.shape)

In [ ]:
display_rand_images(images, labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = random_seed)

print("x_train shape = ",x_train.shape)
print("y_train shape = ",y_train.shape)
print("\nx_test shape = ",x_test.shape)
print("y_test shape = ",y_test.shape)

In [ ]:
display_rand_images(x_train, y_train)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

model = Sequential()

# 1st Conv layer (input shape defined here)
model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), padding="valid", activation="relu", input_shape=(224,224,3)))

# 1st Max Pool layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(BatchNormalization())

# 2nd Conv layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding="valid", activation="relu"))

# 2nd Max Pool layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(BatchNormalization())

# 3rd Conv layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", activation="relu"))

# 4th Conv layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding="valid", activation="relu"))

# 5th Conv layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding="valid", activation="relu"))

# 3rd Max Pool layer
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))

model.add(BatchNormalization())

model.add(Flatten())

# 1st Dense layer (no input_shape here)
model.add(Dense(4096, activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

# 2nd Dense layer
model.add(Dense(4096, activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

# 3rd Dense layer
model.add(Dense(1000, activation="relu"))

model.add(Dropout(0.4))

model.add(BatchNormalization())

# Output layer
model.add(Dense(20, activation="softmax"))

model.summary()


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Train my own custom model

In [ ]:
# checkpoints don't work.
model_checkpoint = ModelCheckpoint(
    filepath="./custom.weights.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

custom_filepath = Path("models/custom.h5").resolve().absolute()
if custom_filepath.exists():    
    model = load_model(custom_filepath)

else:
    model.fit(x_train, y_train, epochs=500, callbacks=[model_checkpoint])
    model.save(custom_filepath, save_format="h5")

Setup Resnet and train it.

In [ ]:
from keras.applications import ResNet50V2

resnet = ResNet50V2()
resnet.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model_checkpoint = ModelCheckpoint(
    filepath="./resnet.weights.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

resnet_filepath = Path("models/resnet.h5").resolve().absolute()
if resnet_filepath.exists():    
    model = load_model(resnet_filepath)

else:
    resnet.fit(x_train, y_train, epochs=100, callbacks=[model_checkpoint])
    resnet.save(resnet_filepath, save_format="h5")

In [ ]:
model_loss, model_accuracy = model.evaluate(x_test, y_test)
resnet_loss, resnet_accuracy = resnet.evaluate(x_test, y_test)

print(f"custom loss: {model_loss} and accuracy: {model_accuracy}")
print(f"resnet loss: {resnet_loss} and accuracy: {resnet_accuracy}")

In [ ]:
model_pred = model.predict(x_test)
resnet_pred = model.predict(x_test)

model_pred.shape
resnet_pred.shape

r = np.random.randint( 0, x_test.shape[0], 1)

In [ ]:
def show_pred(pred, model_name: str, r: int):
    plt.figure(1 , figsize = (19 , 10))
    n = 0

    for i in range(9):
        n += 1

        plt.subplot(3, 3, n)
        plt.subplots_adjust(hspace = 0.3, wspace = 0.3)

        plt.imshow(x_test[r[0]])
        plt.title(f'{model_name}: Actual = {y_test[r[0]]}, Predicted = {(y_test[r[0]]*pred[r[0]][y_test[r[0]]]):.3f}')
        plt.xticks([]) , plt.yticks([])
        plt.bar_label

    plt.show()

In [ ]:
show_pred(model_pred, "custom", r)

In [ ]:
show_pred(resnet_pred, "resnet", r)